# Is the date feature so important?

Recently I learned that  [the test week is the same for all customers](https://www.kaggle.com/c/h-and-m-personalized-fashion-recommendations/discussion/306380).
After that, I noticed that [this recent notebook](https://www.kaggle.com/hengzheng/time-is-our-best-friend) by @hengzheng utilized this point very elegantly.


Now I changed the beginning of data aggregation from 2020-06-01 to 2020-09-01.
The score improved from 0.01 to 0.018 (current public 3rd). I beleave that the seasonal effect is very important in this competition.


In [ ]:
import numpy as np
import pandas as pd

from pathlib import Path

data_path = Path('/kaggle/input/h-and-m-personalized-fashion-recommendations/')

In [ ]:
transactions = pd.read_csv(
    data_path / 'transactions_train.csv',
    # set dtype or pandas will drop the leading '0' and convert to int
    dtype={'article_id': str} 
)

submission = pd.read_csv(data_path / 'sample_submission.csv')

In [ ]:
print(transactions.shape)
transactions.head()

In [ ]:
# only use data after 2020-06-01

transactions['t_dat'] = pd.to_datetime(transactions['t_dat'])
transactions = transactions[transactions['t_dat'] > pd.to_datetime('2020-09-01')]
print(transactions.shape)

In [ ]:
purchase_dict = {}

for i,x in enumerate(zip(transactions['customer_id'], transactions['article_id'])):
    cust_id, art_id = x
    if cust_id not in purchase_dict:
        purchase_dict[cust_id] = {}
    
    if art_id not in purchase_dict[cust_id]:
        purchase_dict[cust_id][art_id] = 0
    
    purchase_dict[cust_id][art_id] += 1
    
print(len(purchase_dict))

In [ ]:
print(submission.shape)
submission.head()

In [ ]:
not_so_fancy_but_fast_benchmark = submission[['customer_id']]
prediction_list = []
dummy_list = list((transactions['article_id'].value_counts()).index)[:12]
dummy_pred = ' '.join(dummy_list)

for i, cust_id in enumerate(submission['customer_id'].values.reshape((-1,))):
    if cust_id in purchase_dict:
        l = sorted((purchase_dict[cust_id]).items(), key=lambda x: x[1], reverse=True)
        l = [y[0] for y in l]
        if len(l)>12:
            s = ' '.join(l[:12])
        else:
            s = ' '.join(l+dummy_list[:(12-len(l))])
    else:
        s = dummy_pred
    prediction_list.append(s)

not_so_fancy_but_fast_benchmark['prediction'] = prediction_list
print(not_so_fancy_but_fast_benchmark.shape)
not_so_fancy_but_fast_benchmark.head()

In [ ]:
not_so_fancy_but_fast_benchmark.to_csv('not_so_fancy_but_fast_benchmark.csv', index=False)